In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from models.vaes import Base, VAE, IWAE, AIS_VAE, AIWAE, ULA_VAE
from models.samplers import HMC, MALA, ULA, run_chain
import yaml
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

%matplotlib widget

In [2]:
pl.__version__

'1.0.4'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def generate_dataset(N, eps0, d=2, sigma=1.):
    z = np.random.randn(N, d)
    x = eps0 * z**2 + np.random.randn(N, d) * sigma
    return x

In [5]:
class Toy(Base):
    def joint_logdensity(self, ):
        def density(z, x):
            z = z.clone()
            x_reconst = self(z)
            log_Pr = torch.distributions.Normal(loc=torch.tensor(0., device=x.device, dtype=torch.float32),
                                                scale=torch.tensor(1., device=x.device, dtype=torch.float32)).log_prob(
                z).sum(-1)
            return -F.mse_loss(x_reconst.view(x_reconst.shape[0], -1),
                                                       x.view(x_reconst.shape[0], -1), reduction='none').sum(
                -1) + log_Pr

        return density

class VAE_Toy(VAE, Toy):
    def loss_function(self, recon_x, x, mu, logvar):
        batch_size = mu.shape[0] // self.num_samples
        MSE = F.mse_loss(recon_x.view(mu.shape[0], -1), x.view(mu.shape[0], -1),
                                                 reduction='none').view(
            (self.num_samples, batch_size, -1)).mean(0).sum(-1).mean()
        KLD = -0.5 * torch.mean((1 + logvar - mu.pow(2) - logvar.exp()).view(
            (self.num_samples, -1, self.hidden_dim)).mean(0).sum(-1))
        loss = MSE + KLD
        return loss
    
class IWAE_Toy(IWAE, Toy):
    def loss_function(self, recon_x, x, mu, logvar, z):
        batch_size = mu.shape[0] // self.num_samples
        log_Q = torch.distributions.Normal(loc=mu,
                                           scale=torch.exp(0.5 * logvar)).log_prob(z).view(
            (self.num_samples, -1, self.hidden_dim)).sum(-1)

        log_Pr = torch.sum((-0.5 * z ** 2).view((self.num_samples, -1, self.hidden_dim)), -1)
        MSE = F.mse_loss(recon_x.view(mu.shape[0], -1), x.view(mu.shape[0], -1),
                                                 reduction='none').view(
            (self.num_samples, batch_size, -1)).sum(-1)
        log_weight = log_Pr - MSE - log_Q
        log_weight = log_weight - torch.max(log_weight, 0)[0]  # for stability
        weight = torch.exp(log_weight)
        weight = weight / torch.sum(weight, 0)
        weight = weight.detach()
        loss = torch.mean(torch.sum(weight * (-log_Pr + MSE + log_Q), 0))

        return loss
    
class ULA_VAE_Toy(ULA_VAE, Toy):
    pass

class AIS_VAE_Toy(AIS_VAE, Toy):
    pass

In [6]:
class ToyDataset(Dataset):
    def __init__(self, data):
        super(ToyDataset, self).__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        sample = torch.tensor(self.data[item], dtype=torch.float32, device=device)
        return sample, -1.

In [21]:
class ToyEncoder(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, d),
            nn.LeakyReLU(),
            nn.Linear(d, 2*d),
        )

    def forward(self, x):
        return self.net(x)
    
class ToyDecoder(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, d),
            nn.LeakyReLU(),
            nn.Linear(d, d),
        )

    def forward(self, x):
        return self.net(x) 
    
class TrueDecoder(nn.Module):
    def __init__(self, d, eps, sigma):
        super().__init__()
        self.eps = nn.Parameter(torch.tensor(eps, dtype=torch.float32), requires_grad=False)
        self.sigma = sigma

    def forward(self, z):
        return self.eps * z**2 + torch.randn_like(z) * self.sigma

In [22]:
N = 10000
d = 2
sigma = 1
eps = 2 + np.random.randn(1, d)
X_train = generate_dataset(N=N, eps0=eps, d=d, sigma=sigma)
X_val = generate_dataset(N=N // 100, eps0=eps, d=d, sigma=sigma)

In [23]:
plt.close()
plt.figure()
plt.title('True data')
plt.hist(x=X_train, bins=100)
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.close()
plt.figure()
plt.title('True data')
plt.scatter(x=X_train[:, 0], y=X_train[:, 1], alpha=0.25)
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
train_dataset = ToyDataset(data=X_train)
val_dataset = ToyDataset(data=X_val)

In [26]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [27]:
def replace_enc_dec(model):
    model.encoder_net = ToyEncoder(d=d)
    model.decoder_net = TrueDecoder(d=d, eps=eps, sigma=1)
    model = model.to(device)
    return model

In [100]:
# ----- VAE ------ #
vae = VAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy')
vae = replace_enc_dec(vae)

# ----- IWAE ------ #
iwae = IWAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=10, hidden_dim=d,
            net_type='conv', dataset='toy')
iwae = replace_enc_dec(iwae)
iwae.name = 'IWAE'

# ----- ULA_VAE ----- #
ula_vae = ULA_VAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy',
            step_size=0.05, K=5, use_transforms=False)
ula_vae = replace_enc_dec(ula_vae)
ula_vae.name = 'ULA_VAE'

# ----- AIS_VAE ----- #
ais_vae = AIS_VAE_Toy(shape=28, act_func=nn.LeakyReLU,
            num_samples=1, hidden_dim=d,
            net_type='conv', dataset='toy',
            step_size=0.05, K=5, use_barker=False)
ais_vae = replace_enc_dec(ais_vae)
ais_vae.name = 'AIS_VAE'
ais_vae.epsilon_target = 0.75

In [29]:
def run_trainer(model):
    tb_logger = pl_loggers.TensorBoardLogger('lightning_logs/')
    trainer = pl.Trainer(logger=tb_logger, fast_dev_run=False, max_epochs=51, track_grad_norm=2.)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)

In [ ]:
run_trainer(ais_vae)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 18    
1 | decoder_net     | TrueDecoder | 2     
2 | transitions_nll | ModuleList  | 8     
3 | transitions     | ModuleList  | 5     
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [33]:
run_trainer(ula_vae)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 18    
1 | decoder_net     | TrueDecoder | 2     
2 | transitions_nll | ModuleList  | 8     
3 | transitions     | ModuleList  | 5     


In [87]:
run_trainer(iwae)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 18    
1 | decoder_net     | TrueDecoder | 2     
2 | transitions_nll | ModuleList  | 8     
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [31]:
run_trainer(vae)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type        | Params
------------------------------------------------
0 | encoder_net     | ToyEncoder  | 18    
1 | decoder_net     | TrueDecoder | 2     
2 | transitions_nll | ModuleList  | 8     


In [35]:
def plot_generated_data(model):
    z = torch.randn(10000, d, dtype=torch.float32, device=device)
    with torch.no_grad():
        generated_samples = model(z).cpu().numpy()

    plt.close()
    plt.figure()
    plt.title('Generated data')
    plt.hist(x=generated_samples, bins=100)
    plt.show();
    
    plt.figure()
    plt.title('Generated data')
    plt.scatter(x=generated_samples[:, 0], y=generated_samples[:, 1], alpha=0.25)
    plt.show();

In [92]:
plot_generated_data(ais_vae)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plot_generated_data(ula_vae)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/matplotlib/axes/_axes.py:6628: RuntimeWarning: All-NaN slice encountered
  xmin = min(xmin, np.nanmin(xi))
/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.8/site-packages/matplotlib/axes/_axes.py:6629: RuntimeWarning: All-NaN slice encountered
  xmax = max(xmax, np.nanmax(xi))


ValueError: autodetected range of [nan, nan] is not finite

In [38]:
plot_generated_data(iwae)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plot_generated_data(vae)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
n_leapfrogs = 5
step_size = 0.05
n_samples = 1000
hmc = HMC(n_leapfrogs=n_leapfrogs, step_size=step_size, partial_ref=False, use_barker=False).to(device)

idx = 2
X_item = torch.tensor(X_val[idx][None], device=device, dtype=torch.float32).repeat(n_samples, 1)

In [94]:
## Receive true posterior sample:
target_density = lambda z, x: torch.distributions.Normal(loc=torch.tensor(eps, device=device) * (z ** 2), scale=sigma).log_prob(X_item).sum(
    -1) + torch.distributions.Normal(loc=torch.tensor(0., device=device, dtype=torch.float32),
                                     scale=torch.tensor(1., device=device, dtype=torch.float32)).log_prob(z).sum(-1)
true_posterior_samples = run_chain(kernel=hmc, z_init=torch.randn(n_samples, 2, device=device), target=target_density, return_trace=False, n_steps=1, burnin=500).cpu().numpy()

In [95]:
def receive_model_samples(model):
    with torch.no_grad():
        x = torch.tensor(X_val[idx][None], device=device, dtype=torch.float32)
        model_samples, mu, logvar = model.enc_rep(x=x, n_samples=n_samples)
        if model.name in ['ULA_VAE', 'AIS_VAE']:
            model_samples = model.run_transitions(z=model_samples, x=x.repeat(n_samples, 1), mu=mu, logvar=logvar, inference_part=False)[0]
    return model_samples.cpu().numpy()

def receive_posterior_samples(model):
    with torch.no_grad():
        model_target_density = lambda z, x: model.joint_logdensity()(z=z, x=X_item)
        model_posterior_samples = run_chain(kernel=hmc, z_init=torch.randn(n_samples, 2, device=device), target=model_target_density, return_trace=False, n_steps=1, burnin=500).cpu().numpy()
    return model_posterior_samples

In [96]:
vae_sample = receive_model_samples(vae)
vae_posterior_sample = receive_posterior_samples(vae)

iwae_sample = receive_model_samples(iwae)
iwae_posterior_sample = receive_posterior_samples(iwae)

ula_vae_sample = receive_model_samples(ula_vae)
ula_vae_posterior_sample = receive_posterior_samples(ula_vae)

ais_vae_sample = receive_model_samples(ais_vae)
ais_vae_posterior_sample = receive_posterior_samples(ais_vae)

In [99]:
# plot
# ========================================
graph = sns.jointplot(x=true_posterior_samples[:, 0], y=true_posterior_samples[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=vae_posterior_sample[:, 0], y=vae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=iwae_posterior_sample[:, 0], y=iwae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=ula_vae_posterior_sample[:, 0], y=ula_vae_posterior_sample[:, 1], color='r', alpha=0.5);
# graph = sns.jointplot(x=ais_vae_posterior_sample[:, 0], y=ais_vae_posterior_sample[:, 1], color='r', alpha=0.5);

# graph.x = vae_sample[:, 0]
# graph.y = vae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='g', s=50, alpha=0.5);

# graph.x = iwae_sample[:, 0]
# graph.y = iwae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='b', s=50, alpha=0.5);

# graph.x = ula_vae_sample[:, 0]
# graph.y = ula_vae_sample[:, 1]
# graph.plot_joint(plt.scatter, marker='x', c='y', s=50, alpha=0.5);

graph.x = ais_vae_sample[:, 0]
graph.y = ais_vae_sample[:, 1]
graph.plot_joint(plt.scatter, marker='x', c='purple', s=50, alpha=0.5);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Block with interpolation. Trained models are required.

In [18]:
train_loader, val_loader = make_dataloaders(dataset='celeba', batch_size=10, val_batch_size=10, binarize=False)

In [19]:
version = 2
epoch = 29

In [20]:
with open(f'lightning_logs/default/version_{version}/hparams.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    fruits_list = yaml.load(file, Loader=yaml.FullLoader)

    print(fruits_list)
    hparams = fruits_list

{'act_func': <class 'torch.nn.modules.activation.LeakyReLU'>, 'dataset': 'celeba', 'hidden_dim': 64, 'name': 'VAE', 'net_type': 'conv', 'num_samples': 1, 'shape': 64}


In [21]:
model = VAE(**hparams).to(device)
checkpoint = torch.load(f'lightning_logs/default/version_{version}/checkpoints/epoch={epoch}.ckpt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [22]:
for batch in train_loader:
    x, _ = batch
    break

In [23]:
plt.close()
plt.figure()
obj1 = x[0][None].to(device)
if obj1.shape[1] == 1:
    plt.imshow(obj1[0][0].cpu())
else:
    plt.imshow(obj1[0].permute((1, 2, 0)).cpu())
plt.tight_layout()
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.close()
plt.figure()
obj2 = x[-1][None].to(device)
if obj2.shape[1] == 1:
    plt.imshow(obj2[0][0].cpu())
else:
    plt.imshow(obj2[0].permute((1, 2, 0)).cpu())
plt.tight_layout()
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Interpolation functions

In [30]:
def interpolate_annealing(model, obj1, obj2, kernel, T=10, n_steps=10):
    with torch.no_grad():
        all_z = torch.tensor([], dtype=torch.float32, device=obj1.device)
        z_init = torch.mean(model.enc_rep(obj1)[0], 0)[None]
        for t in np.linspace(0., 1., T):
            target_density = lambda t: lambda z, x: (1 - t) * model.joint_density()(z=z, x=obj1) + t * model.joint_density()(z=z, x=obj2)
            current_z = run_chain(kernel=kernel, z_init=z_init, target=target_density(t=t), return_trace=False, n_steps=n_steps)
            all_z = torch.cat([all_z, z_init])
            z_init = current_z
    return all_z

def interpolate_mixture(model, obj1, obj2, kernel, T=10, n_steps=10):
    with torch.no_grad():
        all_z = torch.tensor([], dtype=torch.float32, device=obj1.device)
        z_init = torch.mean(model.enc_rep(obj1)[0], 0)[None]
        for t in np.linspace(0.01, 0.99, T):
            target_density = lambda t: lambda z, x: torch.logsumexp(torch.cat([np.log(1 - t) + model.joint_density()(z=z, x=obj1)[None],
                                                                               np.log(t) + model.joint_density()(z=z, x=obj2)[None]]), dim=0)
            current_z = run_chain(kernel=kernel, z_init=z_init, target=target_density(t=t), return_trace=False, n_steps=n_steps)
            all_z = torch.cat([all_z, z_init])
            z_init = current_z
    return all_z

def interpolate_linear(model, obj1, obj2, T=10):
    with torch.no_grad():
        all_z = torch.tensor([], dtype=torch.float32, device=obj1.device)
        z_1 = torch.mean(model.enc_rep(obj1)[0], 0)[None]
        z_2 = torch.mean(model.enc_rep(obj2)[0], 0)[None]
        for t in np.linspace(0., 1., T):
            current_z = (1 - t) * z_1 + t * z_2
            all_z = torch.cat([all_z, current_z])
    return all_z

def visualize(model, z, shape=(-1, 1, 28, 28)):
    with torch.no_grad():
        x_hat = torch.sigmoid(model(z)).view(shape).cpu()
        plt.close()
        plt.figure()
        if shape[1] == 1:
            grid = torchvision.utils.make_grid(x_hat, nrow=15).mean(0)
            plt.imshow(grid, 'gray')
        else:
            grid = torchvision.utils.make_grid(x_hat, nrow=15)
            plt.imshow(grid.permute((1, 2, 0)))
        plt.tight_layout()
        plt.show();

In [26]:
all_z = interpolate_annealing(model, obj1, obj2, hmc, T=10, n_steps=20)
visualize(model, all_z, (-1, 3, 64, 64))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
all_z = interpolate_linear(model, obj1, obj2, T=10)
visualize(model, all_z, (-1, 3, 64, 64))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
all_z = interpolate_mixture(model, obj1, obj2, hmc, T=10, n_steps=30)
visualize(model, all_z, (-1, 3, 64, 64))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …